In [ ]:
import matplotlib.pyplot as plt
from albumentations import Flip, ShiftScaleRotate, RandomRotate90, RandomCrop
from datasets import HearingDataset,HearingDataset_Fast,OceletDataset,OceletDataset_Fast
import os 
import torch
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = "4"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.set_num_threads(3)

import glob,os
from trainval import trainval
import matplotlib.pyplot as plt
from haven import haven_utils as hu

In [ ]:
exp_dict = hu.load_json('Experiment_Exp/exp_config_mynet_1.json')
exp_dict["batch_size"] = 2
exp_dict["max_epoch"] = 15

datadir = '/data114_1/ocelot2023_v0.1.2' 

In [ ]:
import torch 
from torch.utils.data import DataLoader
import albumentations as A
import models
from datasets import OceletDataset
from haven import haven_utils as hu
import tqdm 
import cv2

datadir = '/data114_1/ocelot2023_v0.1.2' 
data_transform = A.Compose([
                # A.Flip(p=0.3),
                # A.Affine(p=0.3),
                # A.Rotate(p=0.3),
                A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=0.3),
                A.GaussianBlur(3, p=0.3),
                A.GaussNoise(30, p=0.3)],
                keypoint_params=A.KeypointParams(format='xy'),
                additional_targets={'mask0': 'mask',
                                    'tissue_img':'mask',
                                    'tissue_mask':'mask',
                                    })
                                    
test_transform = A.Compose([A.Resize(1024, 1024)],
                            keypoint_params=A.KeypointParams(format='xy'),
                            additional_targets={'mask0': 'mask',
                                                'tissue_img':'mask',
                                                'tissue_mask':'mask',
                                                })
test_set = OceletDataset(data_dir=datadir,
                        n_classes=exp_dict["n_classes"],
                        transform=test_transform,
                        option="test")

test_loader = DataLoader(test_set,
                        batch_size=1,
                        num_workers=4)


model_dir = 'Result/v5'
save_dir = 'eval'
model = models.get_model(exp_dict['model'], exp_dict=exp_dict, train_set=test_set).cuda()
model.load_state_dict(hu.torch_load(os.path.join(model_dir, "model_best.pth")))    

In [ ]:
for i, batch in enumerate(tqdm.tqdm(test_loader)):
    cell_images = batch["images"].cuda()
    cell_masks = batch["gt"].long().cuda()
    tissue_img = batch["tissue_img"].cuda()
    tissue_masks = batch["tissue_mask"].cuda()
    roi_loc = batch["roi_loc"].cuda()

    cell_logits,tissue_logits = model.model_base.predict(cell_images,tissue_img,roi_loc)
    prob = cell_logits.sigmoid()
    seg = torch.argmax(prob, dim=1)
    
    cv2.imwrite(os.path.join(save_dir, 'predict/masks', '{}.png'.format(i)),seg[0].detach().cpu().numpy())
    cv2.imwrite(os.path.join(save_dir, 'gt/masks', '{}.png'.format(i)),cell_masks[0].detach().cpu().numpy())
    

In [ ]:
predict = cv2.imread('eval/predict/masks/10.png')[:,:,0]
gt = cv2.imread('eval/gt/masks/10.png')[:,:,0]

fig,axes = plt.subplots(1,2, figsize=(30,10))

axes[0].imshow(predict)
axes[1].imshow(gt)

In [ ]:
!python eval/mask2csv.py
!python eval/convert_pred_csvs_to_json.py -d /data114_1/wzy/homework/code/eval -s test -m predict
!python eval/convert_pred_csvs_to_json.py -d /data114_1/wzy/homework/code/eval -s test -m gt
!python eval/eval.py